In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 5.3 MB/s 
     |████████████████████████████████| 81 kB 8.9 MB/s 
     |████████████████████████████████| 209 kB 58.4 MB/s 
     |████████████████████████████████| 78 kB 7.4 MB/s 
     |████████████████████████████████| 146 kB 63.6 MB/s 
     |████████████████████████████████| 112 kB 54.2 MB/s 
     |████████████████████████████████| 49 kB 5.8 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=4ed474d94a269aa1e6d93ce7cecd8c377619f199524d215f765588c8c338d8d0
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

import joblib
import pickle
import optuna

warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
data=pd.read_excel('/content/drive/MyDrive/교육/AI실무인증과정/학회 발표/data/data_full.xlsx')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1858 entries, 0 to 1857
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             1858 non-null   datetime64[ns]
 1   Account DOW      1858 non-null   object        
 2   REV OBD          1858 non-null   int64         
 3   OBD NET+FSC_KRW  1858 non-null   float64       
 4   OBD A/R_KRW      1858 non-null   float64       
 5   REV CPN          1858 non-null   int64         
 6   CPN NET+FSC_KRW  1858 non-null   float64       
 7   CPN A/R_KRW      1858 non-null   float64       
 8   REV TKT          1858 non-null   int64         
 9   TKT NET+FSC_KRW  1858 non-null   float64       
 10  TKT A/R_KRW      1858 non-null   float64       
 11  WTI              1858 non-null   float64       
 12  exchanges        1858 non-null   float64       
 13  kospi            1858 non-null   float64       
 14  rates            1858 non-null   float64

In [ ]:
data = data[data["Date"].isin(pd.date_range('2016-01-04', '2019-11-30'))]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1427 entries, 3 to 1429
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             1427 non-null   datetime64[ns]
 1   Account DOW      1427 non-null   object        
 2   REV OBD          1427 non-null   int64         
 3   OBD NET+FSC_KRW  1427 non-null   float64       
 4   OBD A/R_KRW      1427 non-null   float64       
 5   REV CPN          1427 non-null   int64         
 6   CPN NET+FSC_KRW  1427 non-null   float64       
 7   CPN A/R_KRW      1427 non-null   float64       
 8   REV TKT          1427 non-null   int64         
 9   TKT NET+FSC_KRW  1427 non-null   float64       
 10  TKT A/R_KRW      1427 non-null   float64       
 11  WTI              1427 non-null   float64       
 12  exchanges        1427 non-null   float64       
 13  kospi            1427 non-null   float64       
 14  rates            1427 non-null   float64

In [ ]:
data.reset_index(drop=True, inplace=True)

In [ ]:
data

,Date,Account DOW,REV OBD,OBD NET+FSC_KRW,OBD A/R_KRW,REV CPN,CPN NET+FSC_KRW,CPN A/R_KRW,REV TKT,TKT NET+FSC_KRW,TKT A/R_KRW,WTI,exchanges,kospi,rates,stock_a,stock_k,stock_kkj
0,2016-01-04,MON,28000,1.154173e+10,412204.5697,36263,1.465623e+10,404164.9715,19528,1.454451e+10,744803.0601,36.76,1189.5,1918.76,1.63,21429,16459,108918
1,2016-01-05,TUE,24657,1.008791e+10,409129.4946,38432,1.538667e+10,400360.8701,20461,1.555175e+10,760067.7182,35.97,1189.5,1930.53,1.64,20881,16186,108918
2,2016-01-06,WED,26920,1.009247e+10,374905.8415,41478,1.597113e+10,385050.5604,21953,1.647945e+10,750669.3999,33.97,1200.0,1925.43,1.64,20785,16063,110383
3,2016-01-07,THU,26624,1.060220e+10,398219.6596,49006,1.798140e+10,366922.4713,25917,1.872443e+10,722476.6906,33.27,1200.2,1904.33,1.64,20809,16003,104522
4,2016-01-08,FRI,28879,1.077601e+10,373143.3549,63847,2.480722e+10,388541.6089,33748,2.598923e+10,770096.9727,33.16,1199.5,1917.62,1.67,20523,15637,105499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1422,2019-11-26,TUE,22295,7.106925e+09,318767.6722,33929,1.501046e+10,442407.9889,18388,1.618648e+10,880274.0396,58.41,1177.5,2121.35,1.48,26679,16567,47670
1423,2019-11-27,WED,25134,8.100108e+09,322276.9318,35642,1.487445e+10,417329.2423,19304,1.609138e+10,833577.7006,58.11,1177.5,2127.85,1.46,26250,16438,46986
1424,2019-11-28,THU,24742,8.116231e+09,328034.5456,35459,1.549214e+10,436902.7653,19087,1.674230e+10,877157.3571,58.11,1179.0,2118.60,1.43,26059,16470,46937
1425,2019-11-29,FRI,29624,9.924881e+09,335028.3726,41190,1.766483e+10,428862.0799,22167,1.915980e+10,864338.9685,55.17,1180.0,2087.96,1.39,25391,16048,47426


In [ ]:
data = data[['Date', 'REV OBD']]

scaler = MinMaxScaler()
scale_cols = ['REV OBD']

# 예측 데이터 사이즈
max_prediction_length = 20

In [ ]:
# 학습용 데이터
data_p = data.iloc[:data.index.max() - max_prediction_length + 1, :]
training_data = scaler.fit_transform(data_p[scale_cols])

In [ ]:
data_p.shape

(1407, 2)

In [ ]:
data_p

,Date,REV OBD
0,2016-01-04,28000
1,2016-01-05,24657
2,2016-01-06,26920
3,2016-01-07,26624
4,2016-01-08,28879
...,...,...
1402,2019-11-06,25486
1403,2019-11-07,25516
1404,2019-11-08,30883
1405,2019-11-09,27640


In [ ]:
training_data.shape

(1407, 1)

In [ ]:
training_data

array([[0.34625858],
       [0.21804165],
       [0.30483642],
       ...,
       [0.45683274],
       [0.33245119],
       [0.34460937]])

In [ ]:
# max_prediction_length 만큼의 데이터는 예측 데이터와 비교를 위해 분리
actual_data = data.loc[~data.index.isin(data_p.index)][scale_cols]
actual_data.shape

(20, 1)

In [ ]:
def sliding_windows(data, lookback_length, forecast_length):

    x = []
    y = []
    
    for i in range(lookback_length, len(data) - forecast_length + 1):
        _x = data[(i-lookback_length) : i]
        _y = data[i : (i + forecast_length)]
        x.append(_x)
        y.append(_y)
    return np.array(x), np.array(y)


def get_data_loader(X, y, batch_size):

    x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

    train_ds = TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
    train_dl = DataLoader(train_ds, batch_size = batch_size)

    val_ds = TensorDataset(torch.Tensor(x_val), torch.Tensor(y_val))
    val_dl = DataLoader(val_ds, batch_size = batch_size)

    input_size = x_train.shape[-1]

    return train_dl, val_dl, input_size

In [ ]:
class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size,
                            num_layers = num_layers, batch_first = True)
        
        self.fc = nn.Linear(hidden_size , num_classes)
        
    def forward(self, x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out

In [ ]:
def train(log_interval, model, train_dl, val_dl, optimizer, criterion, epoch):

    best_loss = np.inf
    for epoch in range(epoch):
        train_loss = 0.0
        model.train()
        for data, target in train_dl:

            if torch.cuda.is_available():
                data, target = data.cuda(), target.cuda()
                model = model.cuda()

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target) # mean-squared error for regression
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # validation
        valid_loss = 0.0
        model.eval()
        for data, target in val_dl:

            if torch.cuda.is_available():
                data, target = data.cuda(), target.cuda()

            output = model(data)         
            loss = criterion(output, target)
            valid_loss += loss.item()

        if ( epoch % log_interval == 0 ):
            print(f'\n Epoch {epoch} \t Training Loss: {train_loss / len(train_dl)} \t Validation Loss: {valid_loss / len(val_dl)} \n')

        if best_loss > (valid_loss / len(val_dl)):
            print(f'Validation Loss Decreased({best_loss:.6f}--->{(valid_loss / len(val_dl)):.6f}) \t Saving The Model')
            best_loss = (valid_loss / len(val_dl))
            torch.save(model.state_dict(), 'lstm_saved_model.pth')

    return best_loss


def smape(a, f):
    return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)

In [ ]:
def objective(trial):

    cfg = { 'device' : "cuda" if torch.cuda.is_available() else "cpu",
            'batch_size' : trial.suggest_categorical('batch_size',[16, 32, 64, 128, 256, 512, 1024]), # [64, 128, 256]
            'num_epochs' : trial.suggest_int('num_epochs', 10, 100, 10), #1000, 1600, 100), 
            'learning_rate' : trial.suggest_loguniform('learning_rate', 1e-3, 1e-1), #trial.suggest_loguniform('learning_rate', 1e-2, 1e-1), # learning rate을 0.01-0.1까지 로그 uniform 분포로 사용          
            'momentum': trial.suggest_uniform('momentum', 0.4, 0.99),
            'optimizer': trial.suggest_categorical('optimizer',[optim.SGD, optim.Adam]), # trial.suggest_categorical('optimizer',[optim.SGD, optim.Adam]),
            # 'activation': trial.suggest_categorical('activation',[ torch.nn.relu6, torch.nn.tanh ]),
            'hidden_size': trial.suggest_categorical('hidden_size',[16, 32, 64, 128, 256, 512, 1024]),
            'num_layers': trial.suggest_int('num_layers', 1, 5, 1),       
            'num_classes': trial.suggest_int('num_classes', 1, 1) }

    torch.manual_seed(42)

    lookback_length = 60
    forecast_length = 1
    log_interval = 100

    x, y = sliding_windows(training_data, lookback_length, forecast_length)

    train_dl, val_dl, input_size = get_data_loader(x, y,  cfg['batch_size'])
    model = LSTM(num_classes=cfg['num_classes'], input_size=input_size, hidden_size=cfg['hidden_size'], num_layers=cfg['num_layers'])
    optimizer = cfg['optimizer'](model.parameters(), lr=cfg['learning_rate'])
    criterion = torch.nn.MSELoss()

    # if torch.cuda.is_available():
    #     model = model.cuda()

    best_loss = train(log_interval, model, train_dl, val_dl, optimizer, criterion, cfg['num_epochs'])

    predict_data = []

    x_pred = x[-1:, :, :]  # last observed input sequence
    y_pred = y[-1]         # last observed target value

    for i in range(max_prediction_length):

        # feed the last forecast back to the model as an input
        x_pred = np.append(x_pred[:, 1:, :], y_pred.reshape(1, 1, 1), axis=1)

        xt_pred = torch.Tensor(x_pred)

        # generate the next forecast
        yt_pred = model(xt_pred)

        # tensor to array
        x_pred = xt_pred.detach().numpy()
        y_pred = yt_pred.detach().numpy()

        # save the forecast
        predict_data.append(y_pred.flatten()[0])

    # transform the forecasts back to the original scale
    predict_data = np.array(predict_data).reshape(-1, 1)
    predict = scaler.inverse_transform(predict_data)

    actual = np.array(actual_data)

    SMAPE = smape(actual, predict)
    print(f' \nSMAPE : {SMAPE}')


    return SMAPE

In [ ]:
sampler = optuna.samplers.TPESampler()
#   sampler = optuna.samplers.SkoptSampler()

# model.load_state_dict(torch.load('lstm_saved_model.pth'))
    
study = optuna.create_study(sampler=sampler, direction='minimize')
study.optimize(objective, n_trials=2)

[I 2022-06-12 13:13:07,950] A new study created in memory with name: no-name-3a1dd6e4-f98b-4198-a809-301e4f80b4e0



 Epoch 0 	 Training Loss: 0.10984184253303443 	 Validation Loss: 0.08607732504606247 

Validation Loss Decreased(inf--->0.086077) 	 Saving The Model
Validation Loss Decreased(0.086077--->0.055353) 	 Saving The Model
Validation Loss Decreased(0.055353--->0.038711) 	 Saving The Model
Validation Loss Decreased(0.038711--->0.029651) 	 Saving The Model
Validation Loss Decreased(0.029651--->0.024685) 	 Saving The Model
Validation Loss Decreased(0.024685--->0.021937) 	 Saving The Model
Validation Loss Decreased(0.021937--->0.020400) 	 Saving The Model
Validation Loss Decreased(0.020400--->0.019527) 	 Saving The Model
Validation Loss Decreased(0.019527--->0.019023) 	 Saving The Model
Validation Loss Decreased(0.019023--->0.018726) 	 Saving The Model
Validation Loss Decreased(0.018726--->0.018546) 	 Saving The Model
Validation Loss Decreased(0.018546--->0.018435) 	 Saving The Model
Validation Loss Decreased(0.018435--->0.018365) 	 Saving The Model
Validation Loss Decreased(0.018365--->0.018319

[W 2022-06-12 15:03:16,402] Trial 0 failed because of the following error: ValueError('cannot reshape array of size 5 into shape (1,1,1)')
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-33-59028c6cd766>", line 40, in objective
    x_pred = np.append(x_pred[:, 1:, :], y_pred.reshape(1, 1, 1), axis=1)
ValueError: cannot reshape array of size 5 into shape (1,1,1)


Validation Loss Decreased(0.018205--->0.018205) 	 Saving The Model


ValueError: ignored